In [2]:
import os
import streamlit as staticmethod
import pickle
import time
import langchain
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import numpy as np

In [18]:
llm = ChatGroq(
    groq_api_key="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",
    model_name="llama-3.1-70b-versatile",
)

loaders = UnstructuredURLLoader(urls=[
    "https://www.reuters.com/markets/us/futures-advance-after-dull-week-gains-tesla-2023-09-11/",
    "https://www.newsbytesapp.com/news/auto/tata-punch-icng-launched-in-india-pricing-features/story"
])
data = loaders.load() 
len(data)

2

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [6]:
# import json

# # Extract data from each Document object
# data_to_save = [{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in docs]

# # Save data to a JSON file
# with open('data.json', 'w') as f:
#     json.dump(data_to_save, f, indent=2)


In [7]:
# import os
# import pickle

# # Specify the correct file path where you saved the vector index
# file_path = "/Users/abhishek/Desktop/Projects/FoodChatbot/vector_index.pkl"

# # Check if the file exists
# if os.path.exists(file_path):
#     with open(file_path, "rb") as f:
#         vectorIndex = pickle.load(f)
#         print("Vector index loaded successfully")
# else:
#     print(f"File not found at {file_path}")


In [8]:
# file_path = "/Users/abhishek/Desktop/Projects/FoodChatbot/vector_index.pkl"

In [9]:
# import os
# import pickle

# file_path = "/Users/abhishek/Desktop/Projects/FoodChatbot/vector_index.pkl"

# if os.path.exists(file_path):
#     with open(file_path, "rb") as f:
#         vectorIndex = pickle.load(f)  # Load as dictionary or list
#         print("Data loaded successfully:", vectorIndex)
# else:
#     print(f"File not found at {file_path}")

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
print("Model loaded successfully")

/Users/abhishek/tensorflow-test/env/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/abhishek/tensorflow-test/env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model loaded successfully


In [21]:
# Create embeddings for each chunk
embeddings = []
for doc in docs:
    embedding = model.encode(doc.page_content)
    embeddings.append(embedding)

In [22]:
import numpy as np
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain.schema import Document

# Create a custom Embeddings class
class CustomEmbeddings(Embeddings):
    def __init__(self, embeddings):
        self.embeddings = embeddings
    
    def embed_documents(self, texts):
        return self.embeddings
    
    def embed_query(self, text):
        # This is a placeholder. In real use, you'd need to embed the query.
        return self.embeddings[0]

# Convert the list of embeddings to a numpy array
embeddings_array = np.array(embeddings)

# Create a list of Document objects (if not already done)
documents = [Document(page_content=doc.page_content, metadata=doc.metadata) for doc in docs]

# Create custom embeddings instance
custom_embeddings = CustomEmbeddings(embeddings_array.tolist())

# Create the FAISS index
vectorstore = FAISS.from_documents(
    documents,
    embedding=custom_embeddings
)

print("FAISS index created successfully")

# Optionally, save the index for later use
vectorstore.save_local("my_faiss_index")
print("FAISS index saved locally")

FAISS index created successfully
FAISS index saved locally


In [23]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorstore, f)

In [24]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [25]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x3955620d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x395562c40>, model_name='llama-3.1-70b-versatile', groq_api_key=SecretStr('**********'))), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t

In [26]:
query = "what is the price of Tata Punch iCNG?"
# query = "what are the main features of punch iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tata Punch iCNG?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Please enable JS and disable any ad blocker",
      "question": "what is the price of Tata Punch iCNG?"
    },
    {
      "context": "Next Article\n\nTata Punch iCNG launched at Rs. 7.1 lakh: Check features\n\nBy\n\nPradnesh Naik\n\n\n                                                    Aug 04, 2023 \n                                                        \n\n01:52 pm\n\nWhat's the story\n\nTata Motors has launched the Punch iCNG in India with starting price tag of Rs. 7.1 lakh (ex-showroom). It is available in five variants. \n \n                           

Token indices sequence length is longer than the specified maximum sequence length for this model (1777 > 1024). Running this sequence through the model will result in indexing errors


[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "what is the price of Tata Punch iCNG?",
  "summaries": "Content: There's no relevant text in the provided portion to answer the question about the price of Tata Punch iCNG.\nSource: https://www.reuters.com/markets/us/futures-advance-after-dull-week-gains-tesla-2023-09-11/\n\nContent: Rs. 7.1 lakh (ex-showroom)\nSource: https://www.newsbytesapp.com/news/auto/tata-punch-icng-launched-in-india-pricing-features/story\n\nContent: There is no relevant text available in the given document to answer the question about the price of Tata Punch i-CNG.\nSource: https://www.newsbytesapp.com/news/auto/tata-punch-icng-launched-in-india-pricing-features/story\n\nContent: There is no relevant text to answer the question about the price of the Tata Punch iCNG.\nSource: https://www.newsbytesapp.com/news/auto/tata-punch-icng-launched-in-india-pricing-features/s

{'answer': 'The price of Tata Punch iCNG is Rs. 7.1 lakh (ex-showroom).\n\n',
 'sources': 'https://www.newsbytesapp.com/news/auto/tata-punch-icng-launched-in-india-pricing-features/story'}